# Corretor Ortográfico utilizando texto de notícias de jornal da web

**Aluno:** Rafhael de Oliveira Martins

De acordo com o [Wikipédia](https://pt.wikipedia.org/wiki/Corretor_ortogr%C3%A1fico), no software, um corretor ortográfico é um recurso de software que verifica erros ortográficos em um texto. Os recursos de verificação ortográfica geralmente são incorporados em software ou serviços, como processador de texto, cliente de e-mail, dicionário eletrônico ou mecanismo de pesquisa.

**Um corretor ortográfico básico realiza os seguintes processos:**

- Ele escaneia o texto e extrai as palavras nele contidas.
- Em seguida, compara cada palavra com uma lista conhecida de palavras escritas corretamente (ou seja, um dicionário). Isso pode conter apenas uma lista de palavras ou também pode conter informações adicionais, como pontos de hifenização ou atributos lexicais e gramaticais.
- Uma etapa adicional é um algoritmo dependente de linguagem para lidar com morfologia. Mesmo para um idioma levemente flexionado como o inglês, o corretor ortográfico precisará considerar diferentes formas da mesma palavra, como plurais, formas verbais, contrações e possessivos. Para muitas outras linguagens, como aquelas que apresentam aglutinação e declinação e conjugação mais complexas, essa parte do processo é mais complicada.

## Referências utilizadas
- https://github.com/ViniViniAntunes/Corretor_Ortografico_NLP
- https://towardsdatascience.com/build-a-spelling-corrector-program-in-python-46bc427cf57f
- https://huggingface.co/oliverguhr/spelling-correction-english-base?text=ze+shop+is+cloed+due+to+covid+19

In [3]:
# Importa Bibliotecas.
import time
import re

import pandas as pd
import numpy as np

import psycopg2
import sqlalchemy  
from sqlalchemy import create_engine

from textblob import Word
from textblob import TextBlob

# from goose3 import Goose

# import spacy
# !spacy download pt_core_news_sm -q

# # fazer a carga
# nlp = spacy.load('pt_core_news_sm')

import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

# Parametros do Pandas, limitando a quantidade máxima e a largura das colunas.
pd.set_option('max_colwidth', 5000)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = "dados/"

## 1 - Base de dados

### 1.1 - URLs de notícias do Jornal Correio Braziliense

In [81]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df_base = pd.read_sql('''select * from tb_url_noticias;''', conn)
print(df_base.shape)

conn.close()

(8635, 6)


In [93]:
# Verificando dados ausentes ou nulos.
df_base.isnull().sum()

ID_NOTICIA       0
DATA_EXTRACAO    0
DESC_JORNAL      0
DESC_TEMA        0
URL_NOTICIA      0
FLAG_EXTRAIDA    0
dtype: int64

In [94]:
# Primeiros dados.
df_base.head()

,ID_NOTICIA,DATA_EXTRACAO,DESC_JORNAL,DESC_TEMA,URL_NOTICIA,FLAG_EXTRAIDA
0,3,2022-09-24,correiobraziliense,home,https://www.correiobraziliense.com.br/politica/2022/09/5039139-tse-identifica-rs-605-milhoes-em-doacoes-suspeitas.html,True
1,4,2022-09-24,correiobraziliense,home,https://www.correiobraziliense.com.br/cidades-df/2022/09/5039108-confira-a-agenda-dos-candidatos-ao-gdf-para-este-sabado.html,True
2,261,2022-10-05,correiobraziliense,opiniao,https://www.correiobraziliense.com.br//opiniao/2022/10/5041502-artigo-o-brasil-mudou.html,True
3,1639,2022-10-13,correiobraziliense,home,https://www.correiobraziliense.com.br/cbforum/testedopezinho.html,True
4,2156,2022-10-17,correiobraziliense,tecnologia,https://www.correiobraziliense.com.br/tecnologia/2022/10/5044373-ia-vence-os-obstaculos.html,True


In [96]:
# Últimos dados.
df_base.tail()

,ID_NOTICIA,DATA_EXTRACAO,DESC_JORNAL,DESC_TEMA,URL_NOTICIA,FLAG_EXTRAIDA
8630,8632,2022-12-08,correiobraziliense,esportes,https://www.correiobraziliense.com.br/esportes/2022/12/5057617-tecnico-da-croacia-faz-jogo-de-esconde-com-tite-sosa-e-o-misterio.html,True
8631,8633,2022-12-08,correiobraziliense,esportes,https://www.correiobraziliense.com.br/esportes/2022/12/5057613-modric-revela-que-o-plano-e-levar-a-croacia-alem-das-quartas-de-final.html,True
8632,8634,2022-12-08,correiobraziliense,economia,https://www.correiobraziliense.com.br/economia/2022/12/5057600-moveinfra-anuncia-rs-783-bilhoes-para-investimento-em-infraestrutura.html,True
8633,8635,2022-12-08,correiobraziliense,economia,https://www.correiobraziliense.com.br/economia/2022/12/5057624-apos-encontro-com-haddad-guedes-reforca-defesa-de-novo-marco-fiscal.html,True
8634,8636,2022-12-08,correiobraziliense,economia,https://www.correiobraziliense.com.br/economia/2022/12/5057740-atraso-na-coleta-do-censo-traz-riscos-a-qualidade-da-informacao-diz-sindicato.html,True


In [97]:
# Notícias por data.
df_base.DATA_EXTRACAO.value_counts()

2022-10-05    304
2022-10-13    296
2022-10-10    295
2022-11-29    197
2022-10-07    195
             ... 
2022-12-03     75
2022-10-23     74
2022-10-16     66
2022-10-29     64
2022-09-24     45
Name: DATA_EXTRACAO, Length: 64, dtype: int64

In [98]:
# notícias por jornal.
df_base.DESC_JORNAL.value_counts()

correiobraziliense    8635
Name: DESC_JORNAL, dtype: int64

In [99]:
# notícias por tema do jornal.
df_base.DESC_TEMA.value_counts()

home                  1668
cidades-df            1090
diversao-e-arte       1049
brasil                 885
esportes               829
politica               758
mundo                  634
economia               555
ciencia-e-saude        300
concursos              218
opiniao                209
euestudante            206
revista-do-correio      67
holofote                64
tecnologia              61
turismo                 40
comunidadeei             2
Name: DESC_TEMA, dtype: int64

### 1.2 - Textos de notícias do Jornal Correio Braziliense

In [13]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df = pd.read_sql('''select * from tb_texto_noticias;''', conn)
print(df.shape)

conn.close()

(8635, 6)


In [14]:
# Primeiros dados.
df.head()

ID_NOTICIA  \
0           1   
1           2   
2           3   
3           4   
4           5   

                                                                                                                                          URL  \
0  https://www.correiobraziliense.com.br/politica/2022/09/5039152-bolsonaro-e-lula-jogam-as-cartas-para-reta-final-da-campanha-eleitoral.html   
1    https://www.correiobraziliense.com.br/politica/2022/09/5039144-disputa-pelo-voto-entre-lula-e-bolsonaro-divide-igrejas-pentecostais.html   
2                      https://www.correiobraziliense.com.br/politica/2022/09/5039139-tse-identifica-rs-605-milhoes-em-doacoes-suspeitas.html   
3               https://www.correiobraziliense.com.br/cidades-df/2022/09/5039108-confira-a-agenda-dos-candidatos-ao-gdf-para-este-sabado.html   
4       https://www.correiobraziliense.com.br/cidades-df/2022/09/5039008-onde-votar-tre-df-anuncia-610-locais-de-votacao-e-67-mil-secoes.html   

               DATA_PUBLICACAO                                AUTOR  \
0  2022-09-24-0303:55:00-10800  {"'Ingrid Soares, Victor Correia'"}   
1  2022-09-24-0303:55:00-10800            {"'Correio Braziliense'"}   
2  2022-09-24-0303:55:00-10800            {"'Correio Braziliense'"}   
3  2022-09-24-0306:00:00-10800            {"'Correio Braziliense'"}   
4  2022-09-23-0317:46:00-10800                    {"'Pedro Marra'"}   

                                                                   TITULO  \
0  Bolsonaro e Lula jogam as cartas para reta final da campanha eleitoral   
1    Disputa pelo voto entre Lula e Bolsonaro divide igrejas pentecostais   
2                      TSE identifica R$ 605 milhões em doações suspeitas   
3                 Confira a agenda dos candidatos ao GDF para este sábado   
4       Onde votar? TRE-DF anuncia 610 locais de votação e 6,7 mil seções   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [15]:
# Últimos dados.
df.tail()

ID_NOTICIA  \
8630        8632   
8631        8633   
8632        8634   
8633        8635   
8634        8636   

                                                                                                                                                    URL  \
8630              https://www.correiobraziliense.com.br/esportes/2022/12/5057617-tecnico-da-croacia-faz-jogo-de-esconde-com-tite-sosa-e-o-misterio.html   
8631          https://www.correiobraziliense.com.br/esportes/2022/12/5057613-modric-revela-que-o-plano-e-levar-a-croacia-alem-das-quartas-de-final.html   
8632           https://www.correiobraziliense.com.br/economia/2022/12/5057600-moveinfra-anuncia-rs-783-bilhoes-para-investimento-em-infraestrutura.html   
8633            https://www.correiobraziliense.com.br/economia/2022/12/5057624-apos-encontro-com-haddad-guedes-reforca-defesa-de-novo-marco-fiscal.html   
8634  https://www.correiobraziliense.com.br/economia/2022/12/5057740-atraso-na-coleta-do-censo-traz-riscos-a-qualidade-da-informacao-diz-sindicato.html   

                  DATA_PUBLICACAO                                       AUTOR  \
8630  2022-12-08-0311:40:00-10800  {"'Marcos Paulo Lima - Enviado especial'"}   
8631  2022-12-08-0311:14:00-10800  {"'Marcos Paulo Lima - Enviado especial'"}   
8632  2022-12-08-0312:04:00-10800                      {"'Michelle Portela'"}   
8633  2022-12-08-0312:17:00-10800                         {"'Rosana Hessel'"}   
8634  2022-12-08-0318:10:00-10800                        {"'Agência Estado'"}   

                                                                              TITULO  \
8630              Técnico da Croácia faz jogo de esconde com Tite; Sosa é o mistério   
8631           Modric revela que o plano é levar a Croácia além das quartas de final   
8632           MoveInfra anuncia R$ 78,3 bilhões para investimento em infraestrutura   
8633            Após encontro com Haddad, Guedes reforça defesa de novo marco fiscal   
8634  Atraso na coleta do Censo traz riscos à qualidade da informação, diz sindicato   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [100]:
# Verificando ausentes ou nulos.
df.isnull().sum()

ID_NOTICIA           0
URL                  0
DATA_PUBLICACAO    147
AUTOR                0
TITULO               0
TEXTO                0
dtype: int64

In [101]:
# # notícias por autor.
df.AUTOR.value_counts()

{"'Correio Braziliense'"}                                                        1153
{"'Agência Estado'"}                                                              782
{"'Observatorio dos Famosos'"}                                                    282
{}                                                                                242
{"'Observatório dos Famosos'"}                                                    196
                                                                                 ... 
{"'Mila Ferreira, Gabriela Ornelas'"}                                               1
{"'Clara Mariz, Hiago Rocha - TV Alterosa e Maicon Costa - Estado de Minas'"}       1
{"'Guy Hedgecoe - Da BBC News em Madri'"}                                           1
{"'Carlos Silva*Isac Mascarenhas*'"}                                                1
{"'Letícia Mori @leticiamori_ - Da BBC News Brasil em São Paulo'"}                  1
Name: AUTOR, Length: 914, dtype: int64

In [104]:
# Maior título de notícia pelo tamanho do título.
tamanho = []
for i in df.TITULO:
    tamanho.append(len(i))
    
max(tamanho)

200

In [105]:
# Maior texto de notícia pelo tamanho do texto.
tamanho = []
for i in df.TEXTO:
    tamanho.append(len(i))
    
max(tamanho)

153350

## 2 - Preparando Corpus

In [16]:
# Salva conteúdo corpus das notícias.
def salva_conteudo_corpus(path, nome_arquivo, texto):
    # Open arquivo '.txt'.
    text_file = open(path+nome_arquivo, "w", encoding='utf-8')
 
    # Write string para o arquivo.
    text_file.write(texto)
 
    # Close arquivo '.txt'.
    text_file.close()

In [36]:
# Junta todo o conteudo das notícias.
conteudo = ''

for linha in range(0, len(df['TEXTO'])):
    conteudo=conteudo+df['TEXTO'][linha]

print("Quantidade de caracteres no corpus: [", len(conteudo), "]")

salva_conteudo_corpus(path, "conteudo_noticias.txt", conteudo)

Quantidade de caracteres no corpus: [ 27273619 ]


In [4]:
# Abrindo o arquivo txt, armazenando o seu conteúdo em uma variável
with open(path+"conteudo_noticias.txt", mode='r', encoding='utf-8') as f:
    conteudo_treino = f.read()

print("Quantidade de caracteres no corpus: [", len(conteudo_treino), "]")
print()
# Mostrando uma parte do conteúdo do arquivo
print(conteudo_treino[:500])

Quantidade de caracteres no corpus: [ 27273619 ]

A oito dias das eleições, os dois principais candidatos à Presidência, Jair Bolsonaro (PL) e Luiz Inácio Lula da Silva (PT), jogam os últimos lances para a semana final antes do primeiro turno de votação, em 2 de outubro. Mas, se na campanha petista a recomendação é de calçar as "sandálias da humildade" e evitar a euforia da cada vez maior possibilidade de vitória no primeiro turno, na do presidente acendeu a luz de emergência.

O comitê de campanha tenta uma correção de rumo por causa das segui


In [5]:
# lowercase - passa o corpus para letras minúsculas.
conteudo_lower = conteudo_treino.lower()
# print(conteudo_lower[:10000])
print("Quantidade de caracteres no corpus: [", len(conteudo_lower), "]")

Quantidade de caracteres no corpus: [ 27273619 ]


In [6]:
# retira pontuações do corpus, permanecendo somentes as palavras.
conteudo_sem_pontuacao = re.sub(r'[^\w\s]', ' ', conteudo_lower)
# print(conteudo[:10000])
print("Quantidade de caracteres no corpus: [", len(conteudo_sem_pontuacao), "]")

Quantidade de caracteres no corpus: [ 27273619 ]


In [7]:
# tokenize o corpus por palavras.
conteudo_tokens = re.findall(r'\w+', conteudo_sem_pontuacao)
# print(conteudo_tokens[:10000])
print("Quantidade total de palvras no corpus: [", len(conteudo_tokens), "]")
print("Quantidade total de palvras unicas no corpus (vocabulário): [", len(set(conteudo_tokens)), "]")

Quantidade total de palvras no corpus: [ 4464813 ]
Quantidade total de palvras unicas no corpus (vocabulário): [ 86481 ]


## 3 - Preparanção do Corretor Ortográfico

### 3.1 -  Palavra com letra faltante (ERRO).

In [25]:
def insere_letras_faltantes(palavra_fatiada):
    """
    Função para inserir letra ou letras faltantes na palavra informada.
    Recebe uma lista de tuplas (esquerdo, direito) que corresponde aos lados
    esquerdo e direito da palavra fatiada em dois.
    """

    # Lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Todas as letras do alfabeto e as vogais acentuadas.
    letras = 'abcedfghijklmnopqrstuvwxyzáâàãéêèíîìóôòõúûùç'

    # Iterando por todas as tuplas da lista recebida
    for esquerdo, direito in palavra_fatiada:
        
        # Iterando por toda letra das variável letras
        for letra in letras:

            # Acrescentando todas as possibilidades de palavras possíveis
            novas_palavras.append(esquerdo + letra + direito)
    
    # Retornando uma lista de possíveis palavras
    return novas_palavras

In [26]:
# Testando a função
print(insere_letras_faltantes([('programa', 'ão')]))

['programaaão', 'programabão', 'programacão', 'programaeão', 'programadão', 'programafão', 'programagão', 'programahão', 'programaião', 'programajão', 'programakão', 'programalão', 'programamão', 'programanão', 'programaoão', 'programapão', 'programaqão', 'programarão', 'programasão', 'programatão', 'programauão', 'programavão', 'programawão', 'programaxão', 'programayão', 'programazão', 'programaáão', 'programaâão', 'programaàão', 'programaãão', 'programaéão', 'programaêão', 'programaèão', 'programaíão', 'programaîão', 'programaìão', 'programaóão', 'programaôão', 'programaòão', 'programaõão', 'programaúão', 'programaûão', 'programaùão', 'programação']


In [27]:
def gera_palavras_1(palavra):
    """
    Função para gerar possíveis palavras de acordo com a palavra passada e fatiada pela lógica.
    """

    # Lista vazia para armazenar as duas fatias de cada palavra.
    fatias = []

    # Iterando por cada letra de cada palavra.
    for i in range(len(palavra) + 1):

        # Armazenando as duas fatias em uma tupla e essa tupla em uma lista.
        fatias.append((palavra[:i], palavra[i:]))

    # Chamando a função 'insere_letras_faltantes' com a lista de tuplas das fatias recém-criadas.
    palavras_geradas = insere_letras_faltantes(fatias)

    # Retornando a lista de possíveis palavras. A palavra correta estará aí no meio.
    return palavras_geradas

In [28]:
# Testando a função
palavras_geradas = gera_palavras_1('programaão')
print("Quantidade de palavra geradas: [", len(palavras_geradas), "]", end="\n\n")
print(palavras_geradas)

Quantidade de palavra geradas: [ 484 ]

['aprogramaão', 'bprogramaão', 'cprogramaão', 'eprogramaão', 'dprogramaão', 'fprogramaão', 'gprogramaão', 'hprogramaão', 'iprogramaão', 'jprogramaão', 'kprogramaão', 'lprogramaão', 'mprogramaão', 'nprogramaão', 'oprogramaão', 'pprogramaão', 'qprogramaão', 'rprogramaão', 'sprogramaão', 'tprogramaão', 'uprogramaão', 'vprogramaão', 'wprogramaão', 'xprogramaão', 'yprogramaão', 'zprogramaão', 'áprogramaão', 'âprogramaão', 'àprogramaão', 'ãprogramaão', 'éprogramaão', 'êprogramaão', 'èprogramaão', 'íprogramaão', 'îprogramaão', 'ìprogramaão', 'óprogramaão', 'ôprogramaão', 'òprogramaão', 'õprogramaão', 'úprogramaão', 'ûprogramaão', 'ùprogramaão', 'çprogramaão', 'parogramaão', 'pbrogramaão', 'pcrogramaão', 'perogramaão', 'pdrogramaão', 'pfrogramaão', 'pgrogramaão', 'phrogramaão', 'pirogramaão', 'pjrogramaão', 'pkrogramaão', 'plrogramaão', 'pmrogramaão', 'pnrogramaão', 'porogramaão', 'pprogramaão', 'pqrogramaão', 'prrogramaão', 'psrogramaão', 'ptrogramaão',

In [29]:
# Mostra a palavra correta que está dentro da lista de possíveis palavra.
for palavra in palavras_geradas:

    # Selecionando a palavra correta
    if palavra == 'programação':

        # Mostrando que a palavra correta está dentro dessa lista
        print("A palavra correta é [", palavra, "]\nEla está dentro da lista na posição [", palavras_geradas.index(palavra), "]")

A palavra correta é [ programação ]
Ela está dentro da lista na posição [ 395 ]


In [30]:
# Calculo da frequência que determinada palavra aparece dentro do corpus. Usando a função FreqDist() da biblioteca nltk. 

# Calcula a frequencia por palavra.
frequencia = nltk.FreqDist(conteudo_tokens)

# Calculando o total de palavras.
total_palavras = len(conteudo_tokens)

# Mostrando as 10 palavras mais comuns da nossa lista_normalizada
frequencia.most_common(10)

[('de', 206086),
 ('a', 146019),
 ('o', 138674),
 ('e', 121862),
 ('que', 99301),
 ('do', 90279),
 ('da', 68367),
 ('em', 64407),
 ('no', 61719),
 ('para', 55206)]

Lembrando que por se tratar de um Corretor Ortográfico, não retiramos as palavras denominadas como `stopwords`: `o, a, os, as, de, da, que, para` e assim por diante. 

In [31]:
def probabilidade_palavra(palavra_gerada):
    """
    Função para calcular a probabilidade de determinada palavra aparecer em nosso corpus.
    """
    # Retorna a probabilidade de determinada palavra aparecer no nosso corpus
    return frequencia[palavra_gerada] / total_palavras

In [32]:
def corretor_palavra_1(palavra_errada):
    """
    Função que recebe uma palavra errada, e retorna a palavra corrigida.
    """
    # Chama a função 'gera_palavras'.
    palavras_geradas = gera_palavras_1(palavra_errada)

    # Selecionando a palavra com maior probabilidade de aparecer em nosso corpus
    # Essa será a palavra correta
    palavra_correta = max(palavras_geradas, key=probabilidade_palavra)

    # Retornando a palavra corrigida
    return palavra_correta

In [36]:
# Testando o corretor.
corretor_palavra_1('programaão')

'programação'

In [37]:
# Testando o corretor.
corretor_palavra_1('test')

'teste'

In [38]:
# Testando o corretor.
corretor_palavra_1('lgica')

'lógica'

### Avaliação do corretor

In [39]:
def cria_dados_teste(nome_arquivo):
    """
    Função que recebe o nome de um arquivo com uma série de palavras digitadas correta e 
    incorretamente para podermos avaliar o nosso corretor.
    """

    # Criando um lista vazia para armazenar as palavras de teste.
    lista_palavras_teste = []

    # Abre o arquivo em mode de leitura.
    f = open(nome_arquivo, 'r', encoding='utf-8')

    # Iterando em toda linha do conteúdo do arquivo de teste.
    for linha in f:
        #print(linha)
        # Separando as palavras digitadas correta e incorretamente avaliarmos.
        correta, errada = linha.split()

        # Armazenando em uma lista as tuplas formadas pelas palavras digitadas correta e incorretamente.
        lista_palavras_teste.append((correta, errada))
    
    # Fechando o arquivo
    f.close()

    # Retornando a lista com as tuplas de palavras digitadas coreta e incorretamante.
    return lista_palavras_teste

In [40]:
# Chamando a função cria_dados_teste() com o nome do arquivo que contém as palavras de teste.
lista_teste = cria_dados_teste('palavras.txt')

In [41]:
def avalia_corretor(testes, corretor_teste):
    """
    Função que recebe uma lista com as tuplas de palavras de teste para avaliar o corretor.
    """

    # Calculando o número de palavras da lista de teste.
    numero_palavras = len(testes)

    # Setando um contador.
    acertou = 0

    # Iterando por cada tupla dentro da lista de teste.
    for correta, errada in testes:

        # Chamando a função corretor() passando cada palavra digitada incorretamente.
        palavra_corrigida = corretor_teste(errada)

        # Conferindo cada palavra para ver se ele conseguiu corrigir.
        if palavra_corrigida == correta:

            # Incrementando o contador.
            acertou += 1
    
    # Calculando a taxa de acerto do nosso corretor.
    taxa_acerto = round(acertou * 100 / numero_palavras, 2)

    # Mostrando a taxa de acerto doe nosso corretor.
    print(f'{taxa_acerto}% de {numero_palavras} palavras')

In [42]:
# Testa corretor.
avalia_corretor(lista_teste, corretor_palavra_1)

2.15% de 186 palavras


### 3.2 -  Palavra com uma letra a mais (ERRO).

In [43]:
def deleta_caracter(fatias):
    """
    Função para deletar um caracter depois que recebe as fatias.
    """

    # Criando uma lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Iterando por todas as tuplas da lista recebida.
    for esquerdo, direito in fatias:

        # Acrescentando todas as possibilidades de palavras possíveis.
        novas_palavras.append(esquerdo + direito[1:])
    
    # Retornando uma lista de possíveis palavras.
    return novas_palavras

In [44]:
# Testando função.
# Criando um exemplo para testarmos a nova função.
exemplo = [('progr', 'samação')]

# Chamando a função deletando_caracter() passando como parâmetro um exemplo.
deleta_caracter(exemplo)

['programação']

In [45]:
def gera_palavras_2(palavra):
    """
    Função para gerar possíveis palavras de acordo com a palavra passada e fatiada pela lógica.
    """

    # Lista vazia para armazenar as duas fatias de cada palavra.
    fatias = []

    # Iterando por cada letra de cada palavra.
    for i in range(len(palavra) + 1):

        # Armazenando as duas fatias em uma tupla e essa tupla em uma lista.
        fatias.append((palavra[:i], palavra[i:]))

    # Chamando a função 'insere_letras_faltantes' com a lista de tuplas das fatias recém-criadas.
    palavras_geradas = insere_letras_faltantes(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += deleta_caracter(fatias)

    # Retornando a lista de possíveis palavras. A palavra correta estará aí no meio.
    return palavras_geradas

In [130]:
# Testa função.
palavras_geradas = gera_palavras_2('progrsamação')

print("Quantidade de palavra geradas: [", len(palavras_geradas), "]", end="\n\n")
print(palavras_geradas)

Quantidade de palavra geradas: [ 585 ]

['aprogrsamação', 'bprogrsamação', 'cprogrsamação', 'eprogrsamação', 'dprogrsamação', 'fprogrsamação', 'gprogrsamação', 'hprogrsamação', 'iprogrsamação', 'jprogrsamação', 'kprogrsamação', 'lprogrsamação', 'mprogrsamação', 'nprogrsamação', 'oprogrsamação', 'pprogrsamação', 'qprogrsamação', 'rprogrsamação', 'sprogrsamação', 'tprogrsamação', 'uprogrsamação', 'vprogrsamação', 'wprogrsamação', 'xprogrsamação', 'yprogrsamação', 'zprogrsamação', 'áprogrsamação', 'âprogrsamação', 'àprogrsamação', 'ãprogrsamação', 'éprogrsamação', 'êprogrsamação', 'èprogrsamação', 'íprogrsamação', 'îprogrsamação', 'ìprogrsamação', 'óprogrsamação', 'ôprogrsamação', 'òprogrsamação', 'õprogrsamação', 'úprogrsamação', 'ûprogrsamação', 'ùprogrsamação', 'çprogrsamação', 'parogrsamação', 'pbrogrsamação', 'pcrogrsamação', 'perogrsamação', 'pdrogrsamação', 'pfrogrsamação', 'pgrogrsamação', 'phrogrsamação', 'pirogrsamação', 'pjrogrsamação', 'pkrogrsamação', 'plrogrsamação', 'pmrogr

In [46]:
# Mostra a palavra correta que está dentro da lista de possíveis palavra.
for palavra in palavras_geradas:

    # Selecionando a palavra correta
    if palavra == 'programação':

        # Mostrando que a palavra correta está dentro dessa lista
        print("A palavra correta é [", palavra, "]\nEla está dentro da lista na posição [", palavras_geradas.index(palavra), "]")

A palavra correta é [ programação ]
Ela está dentro da lista na posição [ 395 ]


In [47]:
def corretor_palavra_2(palavra_errada):
    """
    Função que recebe uma palavra errada, e retorna a palavra corrigida.
    """
    # Chama a função 'gera_palavras'.
    palavras_geradas = gera_palavras_2(palavra_errada)

    # Selecionando a palavra com maior probabilidade de aparecer em nosso corpus
    # Essa será a palavra correta
    palavra_correta = max(palavras_geradas, key=probabilidade_palavra)

    # Retornando a palavra corrigida
    return palavra_correta

### Avaliação do corretor

In [48]:
# Testa corretor.
avalia_corretor(lista_teste, corretor_palavra_2)

45.16% de 186 palavras


### 3.3 -  Palavra com uma letra errada vizinha da letra correta (ERRO - trocando letras).

In [49]:
def troca_caracter(fatias):
    """
    Função que recebe uma lista de tuplas (esquerdo, direito) que corresponde aos lados esquerdo e direito 
    da palavra fatiada em dois.
    """

    # Criando uma lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Todas as letras do alfabeto e as vogais acentuadas.
    letras = 'abcedfghijklmnopqrstuvwxyzáâàãéêèíîìóôòõúûùç'

    # Iterando por todas as tuplas da lista recebida.
    for esquerdo, direito in fatias:

        # Iterando por toda letra das variável letras.
        for letra in letras:

            # Acrescentando todas as possibilidades de palavras possíveis.
            novas_palavras.append(esquerdo + letra + direito[1:])
    
    # Retornando uma lista de possíveis palavras.
    return novas_palavras

In [50]:
# Testando a função
print(troca_caracter([('prog', 'tamação')]))

['progaamação', 'progbamação', 'progcamação', 'progeamação', 'progdamação', 'progfamação', 'proggamação', 'proghamação', 'progiamação', 'progjamação', 'progkamação', 'proglamação', 'progmamação', 'prognamação', 'progoamação', 'progpamação', 'progqamação', 'programação', 'progsamação', 'progtamação', 'proguamação', 'progvamação', 'progwamação', 'progxamação', 'progyamação', 'progzamação', 'progáamação', 'progâamação', 'progàamação', 'progãamação', 'progéamação', 'progêamação', 'progèamação', 'progíamação', 'progîamação', 'progìamação', 'progóamação', 'progôamação', 'progòamação', 'progõamação', 'progúamação', 'progûamação', 'progùamação', 'progçamação']


In [51]:
# Refatorando mais uma vez a função gerador_palavras()
def gera_palavras_3(palavra):
    """
    Função para gerar possíveis palavras de acordo com a palavra passada e fatiada pela lógica.
    """

    # Lista vazia para armazenar as duas fatias de cada palavra.
    fatias = []

    # Iterando por cada letra de cada palavra.
    for i in range(len(palavra) + 1):

        # Armazenando as duas fatias em uma tupla e essa tupla em uma lista.
        fatias.append((palavra[:i], palavra[i:]))

    # Chamando a função 'insere_letras_faltantes' com a lista de tuplas das fatias recém-criadas.
    palavras_geradas = insere_letras_faltantes(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += deleta_caracter(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += troca_caracter(fatias)

    # Retornando a lista de possíveis palavras. A palavra correta estará aí no meio
    return palavras_geradas

In [52]:
# Testa função.
palavras_geradas = gera_palavras_3('progrsamação')

print("Quantidade de palavra geradas: [", len(palavras_geradas), "]", end="\n\n")
print(palavras_geradas)

Quantidade de palavra geradas: [ 1157 ]

['aprogrsamação', 'bprogrsamação', 'cprogrsamação', 'eprogrsamação', 'dprogrsamação', 'fprogrsamação', 'gprogrsamação', 'hprogrsamação', 'iprogrsamação', 'jprogrsamação', 'kprogrsamação', 'lprogrsamação', 'mprogrsamação', 'nprogrsamação', 'oprogrsamação', 'pprogrsamação', 'qprogrsamação', 'rprogrsamação', 'sprogrsamação', 'tprogrsamação', 'uprogrsamação', 'vprogrsamação', 'wprogrsamação', 'xprogrsamação', 'yprogrsamação', 'zprogrsamação', 'áprogrsamação', 'âprogrsamação', 'àprogrsamação', 'ãprogrsamação', 'éprogrsamação', 'êprogrsamação', 'èprogrsamação', 'íprogrsamação', 'îprogrsamação', 'ìprogrsamação', 'óprogrsamação', 'ôprogrsamação', 'òprogrsamação', 'õprogrsamação', 'úprogrsamação', 'ûprogrsamação', 'ùprogrsamação', 'çprogrsamação', 'parogrsamação', 'pbrogrsamação', 'pcrogrsamação', 'perogrsamação', 'pdrogrsamação', 'pfrogrsamação', 'pgrogrsamação', 'phrogrsamação', 'pirogrsamação', 'pjrogrsamação', 'pkrogrsamação', 'plrogrsamação', 'pmrog

In [53]:
# Mostra a palavra correta que está dentro da lista de possíveis palavra.
for palavra in palavras_geradas:

    # Selecionando a palavra correta
    if palavra == 'programação':

        # Mostrando que a palavra correta está dentro dessa lista
        print("A palavra correta é [", palavra, "]\nEla está dentro da lista na posição [", palavras_geradas.index(palavra), "]")

A palavra correta é [ programação ]
Ela está dentro da lista na posição [ 577 ]


In [54]:
def corretor_palavra_3(palavra_errada):
    """
    Função que recebe uma palavra errada, e retorna a palavra corrigida.
    """
    # Chama a função 'gera_palavras'.
    palavras_geradas = gera_palavras_3(palavra_errada)

    # Selecionando a palavra com maior probabilidade de aparecer em nosso corpus
    # Essa será a palavra correta
    palavra_correta = max(palavras_geradas, key=probabilidade_palavra)

    # Retornando a palavra corrigida
    return palavra_correta

### Avaliação do corretor

In [55]:
# Testa corretor.
avalia_corretor(lista_teste, corretor_palavra_3)

79.03% de 186 palavras


### 3.4 -  Palavra com uma letra trocada no lugar da letra correta (ERRO - invertendo letras).

In [56]:
def inverte_caracter(fatias):
    """
    Função que recebe as fatias e inverte os caracteres.
    """

    # Criando uma lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Iterando por todas as tuplas da lista recebida.
    for esquerdo, direito in fatias:
        
        # Selecionando apenas as fatias da direita que têm mais de uma letra, pois, se não, não há o que inverter.
        if len(direito) > 1:
            
            # Acrescentando todas as possibilidades de palavras possíveis.
            novas_palavras.append(esquerdo + direito[1] + direito[0] + direito[2:])
    
    # Retornando uma lista de possíveis palavras.
    return novas_palavras

In [57]:
# Refatorando outra vez a função gerador_palavras()
def gera_palavras_4(palavra):
    """
    Função para gerar possíveis palavras de acordo com a palavra passada e fatiada pela lógica.
    """

    # Lista vazia para armazenar as duas fatias de cada palavra.
    fatias = []

    # Iterando por cada letra de cada palavra.
    for i in range(len(palavra) + 1):

        # Armazenando as duas fatias em uma tupla e essa tupla em uma lista.
        fatias.append((palavra[:i], palavra[i:]))

    # Chamando a função 'insere_letras_faltantes' com a lista de tuplas das fatias recém-criadas.
    palavras_geradas = insere_letras_faltantes(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += deleta_caracter(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += troca_caracter(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += inverte_caracter(fatias)

    # Retornando a lista de possíveis palavras. A palavra correta estará aí no meio
    return palavras_geradas

In [58]:
# Testa função.
palavras_geradas = gera_palavras_4('progrsamação')

print("Quantidade de palavra geradas: [", len(palavras_geradas), "]", end="\n\n")
print(palavras_geradas)

Quantidade de palavra geradas: [ 1168 ]

['aprogrsamação', 'bprogrsamação', 'cprogrsamação', 'eprogrsamação', 'dprogrsamação', 'fprogrsamação', 'gprogrsamação', 'hprogrsamação', 'iprogrsamação', 'jprogrsamação', 'kprogrsamação', 'lprogrsamação', 'mprogrsamação', 'nprogrsamação', 'oprogrsamação', 'pprogrsamação', 'qprogrsamação', 'rprogrsamação', 'sprogrsamação', 'tprogrsamação', 'uprogrsamação', 'vprogrsamação', 'wprogrsamação', 'xprogrsamação', 'yprogrsamação', 'zprogrsamação', 'áprogrsamação', 'âprogrsamação', 'àprogrsamação', 'ãprogrsamação', 'éprogrsamação', 'êprogrsamação', 'èprogrsamação', 'íprogrsamação', 'îprogrsamação', 'ìprogrsamação', 'óprogrsamação', 'ôprogrsamação', 'òprogrsamação', 'õprogrsamação', 'úprogrsamação', 'ûprogrsamação', 'ùprogrsamação', 'çprogrsamação', 'parogrsamação', 'pbrogrsamação', 'pcrogrsamação', 'perogrsamação', 'pdrogrsamação', 'pfrogrsamação', 'pgrogrsamação', 'phrogrsamação', 'pirogrsamação', 'pjrogrsamação', 'pkrogrsamação', 'plrogrsamação', 'pmrog

In [59]:
# Mostra a palavra correta que está dentro da lista de possíveis palavra.
for palavra in palavras_geradas:

    # Selecionando a palavra correta
    if palavra == 'programação':

        # Mostrando que a palavra correta está dentro dessa lista
        print("A palavra correta é [", palavra, "]\nEla está dentro da lista na posição [", palavras_geradas.index(palavra), "]")

A palavra correta é [ programação ]
Ela está dentro da lista na posição [ 577 ]


In [60]:
def corretor_palavra_4(palavra_errada):
    """
    Função que recebe uma palavra errada, e retorna a palavra corrigida.
    """
    # Chama a função 'gera_palavras'.
    palavras_geradas = gera_palavras_4(palavra_errada)

    # Selecionando a palavra com maior probabilidade de aparecer em nosso corpus
    # Essa será a palavra correta
    palavra_correta = max(palavras_geradas, key=probabilidade_palavra)

    # Retornando a palavra corrigida
    return palavra_correta

### Avaliação do corretor

In [61]:
# Testa corretor.
avalia_corretor(lista_teste, corretor_palavra_4)

79.57% de 186 palavras


## 4 - Verificando se o corretor conhece um vocabulário vindo do corpus

In [62]:
def avaliador(testes, vocabulario):
    """
    Função que recebe uma lista com as tuplas de palavras de teste para poder avaliar o nosso corretor.
    """

    # Calculando o número de palavras da lista de teste.
    numero_palavras = len(testes)

    # Setando os contadores.
    acertou = desconhecidas = 0

    # Iterando por cada tupla dentro da lista de teste.
    for correta, errada in testes:

        # Chamando a função corretor() passando cada palavra digitada incorretamente.
        palavra_corrigida = corretor_palavra_4(errada)

        # Incrementando o contador das palavras desconhecidas.
        desconhecidas += (correta not in vocabulario)
        
        # Conferindo cada palavra para ver se ele conseguiu corrigir.
        if palavra_corrigida == correta:

            # Incrementando o contador das palavras corretas.
            acertou += 1
    
    # Calculando a taxa de acerto do nosso corretor.
    taxa_acerto = round(acertou * 100 / numero_palavras, 2)

    # Calculando a taxa de erro referente às palavras desconhecidas.
    taxa_desconhecidas = round(desconhecidas * 100 / numero_palavras, 2)

    # Mostrando a taxa de acerto doe nosso corretor.
    print(f'{taxa_acerto}% de {numero_palavras} das palavras conhecidas\n'
          f'e {taxa_desconhecidas}% das palavras desconhecidas')

In [63]:
# Calculando as palavras conecidas
vocabulario = set(conteudo_tokens)

# Chamando a função avaliador()
avaliador(lista_teste, vocabulario)

79.57% de 186 das palavras conhecidas
e 3.76% das palavras desconhecidas


Corretor Ortográfico está bom pela expectativa de informações disponíveis, acertando `79.57%` das palavras com a base de avaliação/teste, e errando `20.43%`, com isso vemos também que o corretor não conhece cerca de `3.76%` das palavras na base de teste, demonstrando que a base de notícias, não possuem algumas palavras específicas para o modelo.

## 5 - Melhor Corretor

In [13]:
# Calculo da frequência que determinada palavra aparece dentro do corpus. Usando a função FreqDist() da biblioteca nltk. 
# Calcula a frequencia por palavra.
frequencia = nltk.FreqDist(conteudo_tokens)

# Calculando o total de palavras.
total_palavras = len(conteudo_tokens)

# Mostrando as 10 palavras mais comuns da nossa lista_normalizada
frequencia.most_common(10)

[('de', 206086),
 ('a', 146019),
 ('o', 138674),
 ('e', 121862),
 ('que', 99301),
 ('do', 90279),
 ('da', 68367),
 ('em', 64407),
 ('no', 61719),
 ('para', 55206)]

In [14]:
########################### Função insere_letras() ############################
def insere_letras_faltantes(palavra_fatiada):
    """
    Função para inserir letra ou letras faltantes na palavra informada.
    Recebe uma lista de tuplas (esquerdo, direito) que corresponde aos lados
    esquerdo e direito da palavra fatiada em dois.
    """

    # Lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Todas as letras do alfabeto e as vogais acentuadas.
    letras = 'abcedfghijklmnopqrstuvwxyzáâàãéêèíîìóôòõúûùç'

    # Iterando por todas as tuplas da lista recebida
    for esquerdo, direito in palavra_fatiada:
        
        # Iterando por toda letra das variável letras
        for letra in letras:

            # Acrescentando todas as possibilidades de palavras possíveis
            novas_palavras.append(esquerdo + letra + direito)
    
    # Retornando uma lista de possíveis palavras
    return novas_palavras

######################## Função deletando_caracter() ##########################
def deleta_caracter(fatias):
    """
    Função para deletar um caracter depois que recebe as fatias.
    """

    # Criando uma lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Iterando por todas as tuplas da lista recebida.
    for esquerdo, direito in fatias:

        # Acrescentando todas as possibilidades de palavras possíveis.
        novas_palavras.append(esquerdo + direito[1:])
    
    # Retornando uma lista de possíveis palavras.
    return novas_palavras

######################## Função trocando_caracter() ###########################
def troca_caracter(fatias):
    """
    Função que recebe uma lista de tuplas (esquerdo, direito) que corresponde aos lados esquerdo e direito 
    da palavra fatiada em dois.
    """

    # Criando uma lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Todas as letras do alfabeto e as vogais acentuadas.
    letras = 'abcedfghijklmnopqrstuvwxyzáâàãéêèíîìóôòõúûùç'

    # Iterando por todas as tuplas da lista recebida.
    for esquerdo, direito in fatias:

        # Iterando por toda letra das variável letras.
        for letra in letras:

            # Acrescentando todas as possibilidades de palavras possíveis.
            novas_palavras.append(esquerdo + letra + direito[1:])
    
    # Retornando uma lista de possíveis palavras.
    return novas_palavras

####################### Função invertendo_caracter() ##########################
def inverte_caracter(fatias):
    """
    Função que recebe as fatias e inverte os caracteres.
    """

    # Criando uma lista vazia para armazenar as palavras corrigidas.
    novas_palavras = []

    # Iterando por todas as tuplas da lista recebida.
    for esquerdo, direito in fatias:
        
        # Selecionando apenas as fatias da direita que têm mais de uma letra, pois, se não, não há o que inverter.
        if len(direito) > 1:
            
            # Acrescentando todas as possibilidades de palavras possíveis.
            novas_palavras.append(esquerdo + direito[1] + direito[0] + direito[2:])
    
    # Retornando uma lista de possíveis palavras.
    return novas_palavras

######################### Função gerador_palavras() ###########################
# Refatorando outra vez a função gerador_palavras()
def gera_palavras(palavra):
    """
    Função para gerar possíveis palavras de acordo com a palavra passada e fatiada pela lógica.
    """

    # Lista vazia para armazenar as duas fatias de cada palavra.
    fatias = []

    # Iterando por cada letra de cada palavra.
    for i in range(len(palavra) + 1):

        # Armazenando as duas fatias em uma tupla e essa tupla em uma lista.
        fatias.append((palavra[:i], palavra[i:]))

    # Chamando a função 'insere_letras_faltantes' com a lista de tuplas das fatias recém-criadas.
    palavras_geradas = insere_letras_faltantes(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += deleta_caracter(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += troca_caracter(fatias)

    # Acrescentando mais uma função, aqui que o novo corretor começa.
    palavras_geradas += inverte_caracter(fatias)

    # Retornando a lista de possíveis palavras. A palavra correta estará aí no meio
    return palavras_geradas

############################# Função probabilidade_palavra ###############################
def probabilidade_palavra(palavra_gerada):
    """
    Função para calcular a probabilidade de determinada palavra aparecer em nosso corpus.
    """
    # Retorna a probabilidade de determinada palavra aparecer no nosso corpus
    return frequencia[palavra_gerada] / total_palavras

############################# Função corretor_ortografico ###############################
def corretor_ortografico(palavra_errada):
    """
    Função que recebe uma palavra errada, e retorna a palavra corrigida.
    """
    # Chama a função 'gera_palavras'.
    palavras_geradas = gera_palavras(palavra_errada)

    # Selecionando a palavra com maior probabilidade de aparecer em nosso corpus
    # Essa será a palavra correta
    palavra_correta = max(palavras_geradas, key=probabilidade_palavra)

    # Retornando a palavra corrigida
    return palavra_correta

############################# Função avaliador() ###############################
def avaliador(testes, vocabulario):
    """
    Função que recebe uma lista com as tuplas de palavras de teste para poder avaliar o nosso corretor.
    """

    # Calculando o número de palavras da lista de teste.
    numero_palavras = len(testes)

    # Setando os contadores.
    acertou = desconhecidas = 0

    # Iterando por cada tupla dentro da lista de teste.
    for correta, errada in testes:

        # Chamando a função corretor() passando cada palavra digitada incorretamente.
        palavra_corrigida = corretor_ortografico(errada)

        # Incrementando o contador das palavras desconhecidas.
        desconhecidas += (correta not in vocabulario)
        
        # Conferindo cada palavra para ver se ele conseguiu corrigir.
        if palavra_corrigida == correta:

            # Incrementando o contador das palavras corretas.
            acertou += 1
    
    # Calculando a taxa de acerto do nosso corretor.
    taxa_acerto = round(acertou * 100 / numero_palavras, 2)

    # Calculando a taxa de erro referente às palavras desconhecidas.
    taxa_desconhecidas = round(desconhecidas * 100 / numero_palavras, 2)

    # Mostrando a taxa de acerto doe nosso corretor.
    print(f'{taxa_acerto}% de {numero_palavras} das palavras conhecidas\n'
          f'e {taxa_desconhecidas}% das palavras desconhecidas')

In [65]:
# Calculando as palavras conecidas
vocabulario = set(conteudo_tokens)

# Chamando a função avaliador()
avaliador(lista_teste, vocabulario)

79.57% de 186 das palavras conhecidas
e 3.76% das palavras desconhecidas


## 6 - Teste do corretor

In [15]:
# Digite aqui a sua palavra incorreta
teste = 'natrual'

# Mostrando as respostas dos dois corretores
print(f'Entrada =================> {teste}\nResposta do corretor ====> {corretor_ortografico(teste)}')
     

Entrada =================> natrual
Resposta do corretor ====> natural


In [16]:
# Digite aqui a sua palavra incorreta
teste = 'começande'

# Mostrando as respostas dos dois corretores
print(f'Entrada =================> {teste}\nResposta do corretor ====> {corretor_ortografico(teste)}')

Entrada =================> começande
Resposta do corretor ====> começando


In [17]:
# Digite aqui a sua palavra incorreta
teste = 'avaliacão'

# Mostrando as respostas dos dois corretores
print(f'Entrada =================> {teste}\nResposta do corretor ====> {corretor_ortografico(teste)}')

Entrada =================> avaliacão
Resposta do corretor ====> avaliação


## 7 - Modelo com a lib TextBlob

### 7.1 - Corretor de palavra em inglês

In [69]:
word = Word('appple')

result = word.spellcheck()

print(result)

[('apple', 1.0)]


In [70]:
word = Word('pythonn')

result = word.spellcheck()

print(result)

[('pythonn', 0.0)]


### 7.2 - Corretor de frase ou texto em inglês

In [71]:
sentence = TextBlob('A sentencee to checkk!')

result = sentence.correct()

print(result)

A sentence to check!


## 8 - Modelo pré-treinado com método `transforms` disponível no HuggingFaces

In [72]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# tokenizer = AutoTokenizer.from_pretrained("oliverguhr/spelling-correction-english-base")

# model = AutoModelForSeq2SeqLM.from_pretrained("oliverguhr/spelling-correction-english-base")

Downloading: 100%|██████████| 1.74k/1.74k [00:00<00:00, 349kB/s]
C:\Users\Usuario\anaconda3\envs\iesb_tcc\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 558M/558M [02:47<00:00, 3.33MB/s]  


In [74]:
from transformers import pipeline

In [75]:
fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

print(fix_spelling("lets do a comparsion",max_length=2048))

[{'generated_text': "Let's do a comparison."}]


In [76]:
print(fix_spelling('ze shop is cloed due to covid 19',max_length=2048))

[{'generated_text': 'The shop is closed due to Covid 19.'}]


In [77]:
print(fix_spelling('cloed',max_length=2048))

[{'generated_text': 'Closed.'}]
